In [12]:
import numpy as np
import random
from math import exp

In [2]:
def readData(indFile, genoFile, snpFile):
    """
    Function for creating the relevant data types
    """
    snpData, snpDataCols = open(snpFile).read().split(), []
    for index in range(len(snpData) // 6):
        snpDataCols.append(snpData[6 * index: 6 * index + 6])
    snpDataCols.sort(key=lambda x: x[3])
    genoData = open(genoFile, encoding="ISO-8859-1")
    genoData = genoData.read().split()
    indData, indDataCols = open(indFile).read().split(), []
    for index in range(len(indData) // 3):
        indDataCols.append(indData[3 * index: 3 * index + 3])
    return {"ind": indDataCols, "geno": genoData, "snp": snpDataCols}

In [3]:
"""Create CEU"""
ceu = readData("../geneticData/CEU.phind", "../geneticData/CEU.phgeno", "../geneticData/CEU.phsnp")
ceuInd, ceuGeno, ceuSnp =  ceu["ind"], ceu["geno"], ceu["snp"]
print(ceuSnp[1])

['rs1564808', '8', '0.212641', '10000055', 'A', 'G']


In [4]:
"""Create YRI"""
yri = readData("../geneticData/YRI.phind", "../geneticData/YRI.phgeno",  "../geneticData/YRI.phsnp")
yriInd, yriGeno, yriSnp = yri["ind"], yri["geno"], yri["snp"]

['rs3796133', '3', '1.108697', '100000533', 'C', 'T']


In [5]:
"""Create French Yoruba"""
frenchYoruba = readData("../geneticData/French_Yoruba.ind", "../geneticData/French_Yoruba.ind",  
                        "../geneticData/French_Yoruba.snp")
fyInd, fyGeno, fySnp = frenchYoruba["ind"], frenchYoruba["geno"], frenchYoruba["snp"]

In [6]:
def mixedPopulation(genoA, genoB, alpha):
    #Returns a mixed segment
    allB, allA, newGeno = [x for x in range(len(genoB))], [x for x in range(len(genoA))], ""
    for ind in range(len(genoA)):
        if random.uniform(0, 1) < alpha:
            currPop, currGeno = allA, genoA
        else:
            currPop, currGeno = allB, genoB   
        chosenInd = random.choice(currPop)
        currPop.remove(chosenInd)
        numRef = currGeno[chosenInd]
        newGeno += numRef       
    return newGeno

In [7]:
def singlePopulation(geno):
    #Return a segment for a single population
    numInd = len(geno)
    newGeno, allInds = "", [x for x in range(numInd)]
    for ind in range(numInd):
        chosenInd = random.choice(allInds)
        allInds.remove(chosenInd)
        numRef = geno[chosenInd]
        newGeno += numRef
    return newGeno

In [8]:
def singleAdmixture(popA, popB, alpha, numGenerations):
    """
    Model for a single admixture event.
    Takes in dictionaries as created by readData for each population A and B.
    Alpha is the portion of population A.
    NumGenerations is the number of generations since admixture.
    """
    admixedGeno, admixedSnp = [], []
    indA, genoA, snpA = popA["ind"], popA["geno"], popA["snp"]
    indB, genoB, snpB = popB["ind"], popB["geno"], popB["snp"]
    for currPos in range(1, len(snpA)):
        distance = float(snpA[currPos][2]) - float(snpA[currPos - 1][2])
        if np.random.uniform() < exp(- numGenerations * distance):
            #Choosing from pop A or B
            if np.random.uniform() < alpha:
                #Choose pop A
                newGeno = singlePopulation(genoA[currPos])
                admixedSnp.append(snpA[currPos])
                admixedGeno.append(newGeno)
            else:
                #Choose pop B
                newGeno = singlePopulation(genoB[currPos])
                admixedSnp.append(snpB[currPos])
                admixedGeno.append(newGeno)
        else:
            #Mixed population
            newGeno = mixedPopulation(genoA[currPos], genoB[currPos], alpha)
            admixedSnp.append(snpA[currPos])
            admixedGeno.append(newGeno)
    return admixedGeno, admixedSnp

In [9]:
admixedGeno, admixedSnp = singleAdmixture(ceu, yri, 0.5, 10)

In [10]:
# test = readData("../geneticData/CEU.phind", "newtestPop.geno", "newtestPop.snp")
# snpTest = test['snp']
# genoTest = test['geno']
print(admixedSnp[:3])
print(admixedGeno[:3])
print(admixedSnp == yriSnp[1:])
print(admixedGeno == yriGeno[1:])
print(admixedGeno == ceuGeno[1:])

[['rs1564808', '8', '0.212641', '10000055', 'A', 'G'], ['rs4504482', '6', '1.045887', '100000659', 'T', 'C'], ['rs837309', '13', '0.954699', '100000749', 'T', 'C']]
['101101010110101010111110011011011100111100110101111111111111111111111101111100111001101100110101011111111111111011111111111010010110111111010100111011111110111101100111010111101110101011111000011111010111011101111111101111', '111111111100001111011111110111101011101010111011101110110001101000010001101111001001100100110111111101110101100101011010011000110011111110101111110111101111101001001001100011011110100011011010010111000110110011011101110011', '011111110111111001111111111011001111011011100111000011101000101110011111110011110011101111011110011011101111101111011101111010010111011011111001011101111100111011011111110101100101111110111111100101011111010110011110111111']
True
False
False


In [11]:
print(len(yriInd), len(ceuInd))

224 222
